In [13]:
from pyspark import SparkContext

In [14]:
#sc.stop()

In [15]:
sc = SparkContext(master="local[2]")

In [16]:
sc # python 3.12.0 spark 3.5.1

<SparkContext master=local[2] appName=pyspark-shell>

In [17]:
from pyspark.sql import SparkSession

In [18]:
spark = SparkSession.builder \
    .appName("TextClassifierwithPySpark") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .getOrCreate()

spark.sparkContext.setLogLevel("DEBUG")


In [19]:
df = spark.read.csv("trt/karisik_veri.csv",header=True,inferSchema=True)

In [20]:
df.show()

+---------------+--------------------+--------------------+
|       Kategori|              Başlık|              İçerik|
+---------------+--------------------+--------------------+
|           Spor|Hakkarigücü Kadın...|Turkcell Kadın Fu...|
|Bilim Teknoloji|Uzay raporu: Mars...|Hakemli bilimsel ...|
|         Sağlık|Gaziantep Şehir H...|Gaziantep Şehir H...|
|         Sağlık|Türkiye sınırın d...|Fırat Kalkanı Har...|
|        Ekonomi|3,2 milyar liralı...|Aile ve Sosyal Hi...|
|           Spor|Gençler Kılıç Dün...|Eskrimde, Gençler...|
|         Sağlık|"Gıda mühendisind...|"Mutfakta yapılan...|
|           Spor|Michael Schumache...|Formula 1'in ünlü...|
|         Eğitim|"Türkiye ile Güne...|Milli Eğitim Baka...|
|     ÖZEL HABER|7 Güzel Adam'ın b...|7 Güzel Adam'ın b...|
|         Sağlık|DSÖ, COVID-19'un ...|Dünya Sağlık Örgü...|
|           Spor|Kupada son çeyrek...|Ziraat Türkiye Ku...|
|         Eğitim|Meslek liselerind...|Milli Eğitim Baka...|
|   Kültür-Sanat|Dünyaca ünlü İngi...|Dü

In [21]:
df.groupBy('Kategori','İçerik').count().show()

+---------------+--------------------+-----+
|       Kategori|              İçerik|count|
+---------------+--------------------+-----+
|   Kültür-Sanat|Karabük'ün tarihi...|    1|
|   Kültür-Sanat|"TRT Belgesel eki...|    1|
|   Kültür-Sanat|"Ünlü Fransız res...|    1|
|        Ekonomi|"Depremlerden etk...|    1|
|        Ekonomi|Merkez Bankası, P...|    1|
|     ÖZEL HABER|Telafi eğitiminin...|    1|
|         Eğitim|Kayseri İl Tüketi...|    1|
|         Eğitim|Milli Eğitim Baka...|    1|
|   Kültür-Sanat|Turizmin başkenti...|    1|
|         Sağlık|Zayıflama çayları...|    1|
|         Eğitim|Milli Eğitim Baka...|    1|
|   Kültür-Sanat|Karabük'ün Eskipa...|    1|
|Bilim Teknoloji|"Nobel ödüllü bil...|    1|
|           Spor|MKE Ankaragücü, T...|    1|
|   Kültür-Sanat|2 milyon kitaba e...|    1|
|         Sağlık|"COVID-19'un Omic...|    1|
|        Ekonomi|İşsizlik Sigortas...|    1|
|     ÖZEL HABER|Candaş Şişman... ...|    2|
|        Ekonomi|Emtia piyasası, g...|    1|
|        E

In [22]:
import pandas as pd
pd.set_option('display.max_rows', None)  # Tüm satırları göster
df.toPandas()['Kategori'].value_counts()

Kategori
Spor                                                                                                                                                                                                 4974
Eğitim                                                                                                                                                                                               4931
Sağlık                                                                                                                                                                                               4742
Ekonomi                                                                                                                                                                                              4741
Kültür-Sanat                                                                                                                                                                           

In [23]:
df.count()

29861

In [24]:
df.toPandas()['Kategori'].isnull().sum()

0

In [25]:
df.toPandas()['İçerik'].isnull().sum()


169

In [26]:
df.toPandas()['Başlık'].isnull().sum()

91

In [27]:
df = df.dropna(subset=('İçerik'))
df = df.dropna(subset=('Başlık'))

In [28]:
df.toPandas()['İçerik'].isnull().sum()


0

In [29]:
df.count()

29692

In [30]:
from pyspark.sql.functions import col
#filtered_df = df.filter(~col('Kategori').isin('DOSYA HABER'))


In [31]:
#filtered_df.count()

Feature Extraction

In [32]:
import pyspark.ml.feature

In [33]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF
from pyspark.ml.feature import StringIndexer

In [34]:
tokenizer = Tokenizer(inputCol='İçerik',outputCol='mytokens')
stopwords_remover = StopWordsRemover(inputCol='mytokens',outputCol='filtered_tokens')
vectorizer = CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures')
idf = IDF(inputCol='rawFeatures',outputCol='vectorizedFeatures')

In [35]:
labelEncoder = StringIndexer(inputCol='Kategori',outputCol='label').fit(df)

In [36]:
labelEncoder.transform(df).show()

+---------------+--------------------+--------------------+-----+
|       Kategori|              Başlık|              İçerik|label|
+---------------+--------------------+--------------------+-----+
|           Spor|Hakkarigücü Kadın...|Turkcell Kadın Fu...|  0.0|
|Bilim Teknoloji|Uzay raporu: Mars...|Hakemli bilimsel ...|  5.0|
|         Sağlık|Gaziantep Şehir H...|Gaziantep Şehir H...|  3.0|
|         Sağlık|Türkiye sınırın d...|Fırat Kalkanı Har...|  3.0|
|        Ekonomi|3,2 milyar liralı...|Aile ve Sosyal Hi...|  2.0|
|           Spor|Gençler Kılıç Dün...|Eskrimde, Gençler...|  0.0|
|         Sağlık|"Gıda mühendisind...|"Mutfakta yapılan...|  3.0|
|           Spor|Michael Schumache...|Formula 1'in ünlü...|  0.0|
|         Eğitim|"Türkiye ile Güne...|Milli Eğitim Baka...|  1.0|
|     ÖZEL HABER|7 Güzel Adam'ın b...|7 Güzel Adam'ın b...|  6.0|
|         Sağlık|DSÖ, COVID-19'un ...|Dünya Sağlık Örgü...|  3.0|
|           Spor|Kupada son çeyrek...|Ziraat Türkiye Ku...|  0.0|
|         

In [37]:
labelEncoder.labels

['Spor',
 'Eğitim',
 'Ekonomi',
 'Sağlık',
 'Kültür-Sanat',
 'Bilim Teknoloji',
 'ÖZEL HABER',
 'DOSYA HABER',
 'Türkiye',
 'Ardından televizyonlar',
 'Artık marka araçların yeni modelleri bazı özellikler olmadan üretilecek. Kısacası araç almak için sıraya giren müşteriler',
 "Avrupa'da yapılan araştırmada",
 'Açıklanan son haftalık haritada',
 'Bu dönemde',
 "Bugün Yükseköğretim Kurumları Sınavı'nın ilk oturumu",
 'Denemeleri başarılı olmadı belki',
 'Deneyler',
 'Dizinin galası',
 'Dünya Müzikleri Topluluğunu kurdu',
 'Finansal istikrarı güçlendirici adımlar hayata geçiriliyor. Banka',
 'Haritada çarpma havzaları',
 'Hedefler karşılık bulurken',
 'Kripto paralar yatırımlara',
 "Kıraç ''Vatan Marşı'' isimli bestesine klip çekerek desteğini gösterirken",
 'Müzede',
 'NASA’nın uzay aracı Parker',
 'Peki nedir bu dijital ikiz teknolojisi',
 'Sanayi ve Teknoloji Bakanı Mehmet Fatih Kacır',
 'Selçuklulardan kalma mezar taşları',
 'Sesiyle',
 'Yeni açıklanan eylül ayı enflasyon verilerinin 

In [38]:
label_dict = {
 'Spor':0.0 ,
 'Ekonomi':1.0,
 'Eğitim':2.0,
 'Sağlık':3.0,
 'Bilim Teknoloji':4.0,
 'Kültür-Sanat':5.0,
 #'cevre':7.0
}

In [39]:
df.show()
df.count()

+---------------+--------------------+--------------------+
|       Kategori|              Başlık|              İçerik|
+---------------+--------------------+--------------------+
|           Spor|Hakkarigücü Kadın...|Turkcell Kadın Fu...|
|Bilim Teknoloji|Uzay raporu: Mars...|Hakemli bilimsel ...|
|         Sağlık|Gaziantep Şehir H...|Gaziantep Şehir H...|
|         Sağlık|Türkiye sınırın d...|Fırat Kalkanı Har...|
|        Ekonomi|3,2 milyar liralı...|Aile ve Sosyal Hi...|
|           Spor|Gençler Kılıç Dün...|Eskrimde, Gençler...|
|         Sağlık|"Gıda mühendisind...|"Mutfakta yapılan...|
|           Spor|Michael Schumache...|Formula 1'in ünlü...|
|         Eğitim|"Türkiye ile Güne...|Milli Eğitim Baka...|
|     ÖZEL HABER|7 Güzel Adam'ın b...|7 Güzel Adam'ın b...|
|         Sağlık|DSÖ, COVID-19'un ...|Dünya Sağlık Örgü...|
|           Spor|Kupada son çeyrek...|Ziraat Türkiye Ku...|
|         Eğitim|Meslek liselerind...|Milli Eğitim Baka...|
|   Kültür-Sanat|Dünyaca ünlü İngi...|Dü

29692

In [40]:
df = labelEncoder.transform(df)

In [41]:
df.show()

+---------------+--------------------+--------------------+-----+
|       Kategori|              Başlık|              İçerik|label|
+---------------+--------------------+--------------------+-----+
|           Spor|Hakkarigücü Kadın...|Turkcell Kadın Fu...|  0.0|
|Bilim Teknoloji|Uzay raporu: Mars...|Hakemli bilimsel ...|  5.0|
|         Sağlık|Gaziantep Şehir H...|Gaziantep Şehir H...|  3.0|
|         Sağlık|Türkiye sınırın d...|Fırat Kalkanı Har...|  3.0|
|        Ekonomi|3,2 milyar liralı...|Aile ve Sosyal Hi...|  2.0|
|           Spor|Gençler Kılıç Dün...|Eskrimde, Gençler...|  0.0|
|         Sağlık|"Gıda mühendisind...|"Mutfakta yapılan...|  3.0|
|           Spor|Michael Schumache...|Formula 1'in ünlü...|  0.0|
|         Eğitim|"Türkiye ile Güne...|Milli Eğitim Baka...|  1.0|
|     ÖZEL HABER|7 Güzel Adam'ın b...|7 Güzel Adam'ın b...|  6.0|
|         Sağlık|DSÖ, COVID-19'un ...|Dünya Sağlık Örgü...|  3.0|
|           Spor|Kupada son çeyrek...|Ziraat Türkiye Ku...|  0.0|
|         

In [42]:
(trainDF,testDF) = df.randomSplit((0.8,0.2),seed=42)

In [43]:
trainDF.show()

+--------------------+--------------------+--------------------+-----+
|            Kategori|              Başlık|              İçerik|label|
+--------------------+--------------------+--------------------+-----+
|Ardından televizy...| bundan 39 yıl ön...| Yayıncılık Müzes...|  9.0|
|Avrupa'da yapılan...| düzenli olarak k...| sürekli kahve iç...| 11.0|
|     Bilim Teknoloji|"""1 milyon yazıl...|"Cumhurbaşkanı Re...|  5.0|
|     Bilim Teknoloji|"""100 bin dolara...|"NASA JPL öncü mü...|  5.0|
|     Bilim Teknoloji|"""20'li yaşlar c...|"Ulaştırma ve Alt...|  5.0|
|     Bilim Teknoloji|"""Aile Destek"" ...|"Sosyal hizmet ve...|  5.0|
|     Bilim Teknoloji|"""Ateş çemberi""...|2019 yılının son ...|  5.0|
|     Bilim Teknoloji|"""Ateş Şövalyesi...|Ormanın yumuşak d...|  5.0|
|     Bilim Teknoloji|"""BBC 100 Kadın ...|"Sabancı Üniversi...|  5.0|
|     Bilim Teknoloji|"""Bir Adam Yarat...|"Beyoğlu Kültür Y...|  5.0|
|     Bilim Teknoloji|"""Cep""te dayanı...|Bilgi Teknolojile...|  5.0|
|     

In [44]:
from pyspark.ml.classification import LogisticRegression

In [45]:
lr = LogisticRegression(featuresCol='vectorizedFeatures',labelCol='label')

In [46]:
from pyspark.ml import Pipeline

In [47]:
pipeline = Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf,lr])

In [48]:
pipeline

Pipeline_2f2876eb08e6

In [49]:
pipeline.stages

Param(parent='Pipeline_2f2876eb08e6', name='stages', doc='a list of pipeline stages')

In [50]:
lr_model = pipeline.fit(trainDF)#1m 7.9s

In [51]:
lr_model

PipelineModel_ea7ae92e7df3

In [52]:
predictions = lr_model.transform(testDF)

In [53]:
predictions.show()

+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|            Kategori|              Başlık|              İçerik|label|            mytokens|     filtered_tokens|         rawFeatures|  vectorizedFeatures|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Açıklanan son haf...| 68 ilde vaka say...| Ankara ve İzmir’...| 12.0|[, ankara, ve, iz...|[, ankara, ve, iz...|(69721,[0,20,198,...|(69721,[0,20,198,...|[-8.7673890039219...|[3.42965843126392...|       4.0|
|     Bilim Teknoloji|"""2021 savunma s...|"Yerli ve milli s...|  5.0|["yerli, ve, mill...|["yerli, ve, mill...|(69721,[0,1,5,7,1...|(69721,[0,1,5,7,1...|[-3.2027304190

In [54]:
predictions.columns

['Kategori',
 'Başlık',
 'İçerik',
 'label',
 'mytokens',
 'filtered_tokens',
 'rawFeatures',
 'vectorizedFeatures',
 'rawPrediction',
 'probability',
 'prediction']

In [55]:

predictions.select('rawPrediction','probability','Kategori','label','prediction').show(10)

+--------------------+--------------------+--------------------+-----+----------+
|       rawPrediction|         probability|            Kategori|label|prediction|
+--------------------+--------------------+--------------------+-----+----------+
|[-8.7673890039219...|[3.42965843126392...|Açıklanan son haf...| 12.0|       4.0|
|[-3.2027304190674...|[1.28031156538074...|     Bilim Teknoloji|  5.0|       5.0|
|[-31.503056069095...|[2.12619438758917...|     Bilim Teknoloji|  5.0|       5.0|
|[-24.819585238591...|[3.47109795941216...|     Bilim Teknoloji|  5.0|       5.0|
|[-16.278093807331...|[1.25787609518740...|     Bilim Teknoloji|  5.0|       5.0|
|[-19.556413447195...|[4.64274929249533...|     Bilim Teknoloji|  5.0|       5.0|
|[-13.545048356306...|[1.51839682243025...|     Bilim Teknoloji|  5.0|       3.0|
|[-6.3871186105453...|[1.35875502582184...|     Bilim Teknoloji|  5.0|       5.0|
|[81.0084209957655...|[1.0,1.3647159052...|     Bilim Teknoloji|  5.0|       0.0|
|[-8.54118273330

In [56]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [57]:
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')

In [58]:
accuracy = evaluator.evaluate(predictions)

In [59]:
accuracy # başlık : 79.6  \|/ içerik : 85.15 

0.8515611670363419

Test Edelim

In [60]:
from pyspark.sql import Row

data = [
    Row(Başlık="Bakan Mehmet Şimşek'ten 'enflasyon' açıklaması: Mayıs ayında en yüksek seviyeye ulaşacak", İçerik="Hazine ve Maliye Bakanı Mehmet Şimşek, enflasyon ile ilgili, \"Mayıs ayında yıllık enflasyonun en yüksek seviyesine ulaştıktan sonra yılın kalan döneminde hızla gerilemesini bekliyoruz\" değerlendirmesini yaptı.")
]


type(data)

list

In [61]:

biz_test_df = spark.createDataFrame(data)
#test_df.show()

In [62]:
predictions = lr_model.transform(biz_test_df)
print(predictions.columns)

['Başlık', 'İçerik', 'mytokens', 'filtered_tokens', 'rawFeatures', 'vectorizedFeatures', 'rawPrediction', 'probability', 'prediction']


In [65]:
#predictions.collect()
#predictions.show()

In [1]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['kategorilenmisVeri'] 

In [8]:
kategoriler = {
    'bilim-teknoloji': 'bilim-teknoloji.csv',
    'cevre': 'cevre.csv',
    'saglik': 'saglik.csv',
    'spor': 'spor.csv',
    'kultur-sanat': 'kultur-sanat.csv',
    'ekonomi': 'ekonomi.csv',
    'egitim': 'egitim.csv'
}

In [9]:
for kategori, dosya_adi in kategoriler.items():
    collection = db[kategori] 

    df = pd.read_csv(dosya_adi)

    for index, row in df.iterrows():
        data = row.to_dict()
        collection.insert_one(data)